# Fast-Trips Tutorial 4: Your own city!


In [ ]:
import os,datetime,shutil
import pandas as pd

# Specify Input Networks + Visualize

In [ ]:
GTFS_LINK  = r"http://admin.gotransitnc.org/sites/default/files/developergtfs/GoRaleigh_GTFS_0.zip"

BASE_DIR   = r"C:\Users\lzorn\Documents\fast-trips-tutorial"
NEW_FOLDER = "GoRaleigh_GTFS"
GTFS_LOC   = os.path.join(BASE_DIR,NEW_FOLDER)

In [ ]:
# Download the file from the URL and unzip
from urllib import urlopen
from zipfile import ZipFile

try:
    os.stat(os.path.join(BASE_DIR,NEW_FOLDER))
except:
    os.mkdir(os.path.join(BASE_DIR,NEW_FOLDER)) 


tempzip_filename = os.path.join(BASE_DIR,NEW_FOLDER,"tempgtfs.zip")
zipresp = urlopen(GTFS_LINK)
tempzip = open(tempzip_filename, "wb")
tempzip.write(zipresp.read())
tempzip.close()
zf = ZipFile(tempzip_filename)
zf.extractall(path = os.path.join(BASE_DIR,NEW_FOLDER))
zf.close()
os.remove(tempzip_filename)

## Validate GTFS Feed
Make sure you are starting with a valid network.
This can take a while for a large network. 

In [ ]:
import transitfeed
loader        = transitfeed.Loader(GTFS_LOC, memory_db=True)
schedule      = loader.Load()
schedule.Validate()

In [ ]:
print "Routes Loaded:"
rts = [r.route_long_name for r in schedule.routes.itervalues()]
for r in rts:
    print " - ",r

## Create Simple Map of Routes

In [ ]:
import folium
import tutorial_map

mymap = tutorial_map.make_map(schedule)
mymap

## Add needed data to turn GTFS to GTFS-PLUS
There are files that we need to add:  
  * routes_ft.txt  
  * vehicles_ft.txt  
  * trips_ft.txt  
  * transfers_ft.txt 
  * walk_access_ft.txt 

In [ ]:
import csv
import gtfs_plus

In [ ]:
GTFS_PLUS_LOC   = "GoRaleigh_GTFS_PLUS"
OUTPUT_DIR      = os.path.join(BASE_DIR,GTFS_PLUS_LOC)

# start with the GTFS files if you don't have these already
try:
    shutil.copytree(GTFS_LOC, "GoRaleigh_GTFS_PLUS")
    # copy over the config file from the earlier tutorials
    shutil.copy(os.path.join(BASE_DIR,"tta","input","demand-single","config_ft.txt"), 
                os.path.join(OUTPUT_DIR, "config_ft.txt"))
except:
    # hopefully this is ok and you're just doing this multiple times
    pass

DEFAULT_MODE    = "local_bus"
DEFAULT_VEHICLE = "standard_bus"
SEATED_CAPACITY   = 30
STANDING_CAPACITY = 20
MAX_SPEED         = 45
ACCELERATION      = 3
DECELERATION      = 4
DWELL = r'"3 + 2*[boards] + 1.5*[alights]"'

### Create `routes_ft.txt` 
For now, assume a default mode

In [ ]:
route_modes_dict = gtfs_plus.routesft_assume_mode(schedule, DEFAULT_MODE)
with open(os.path.join(OUTPUT_DIR,'routes_ft.txt'),'wb') as f:
    f.write("route_id,mode\n")
    w = csv.writer(f)
    w.writerows(route_modes_dict.items())

### Create `trips_ft.txt` 
For now, assume a default vehicle

In [ ]:
trip_vehicle_dict = dict(zip(schedule.trips.keys(),[DEFAULT_VEHICLE]*len(schedule.trips.keys())))
with open(os.path.join(OUTPUT_DIR,'trips_ft.txt'),'wb') as f:
    f.write("trip_id,vehicle_name\n")
    w = csv.writer(f)
    w.writerows(trip_vehicle_dict.items())

### Create `vehicles_ft.txt` 
FOr now, assume mostly defaults

In [ ]:
with open(os.path.join(OUTPUT_DIR,'vehicles_ft.txt'),'wb') as f:
    f.write("vehicle_name,seated_capacity,standing_capacity,max_speed,acceleration,deceleration,dwell_formula\n")
    f.write("%s,%d,%d,%4.2f,%4.2f,%4.2f,%s\n"%(DEFAULT_VEHICLE,SEATED_CAPACITY,STANDING_CAPACITY,MAX_SPEED,ACCELERATION,DECELERATION,DWELL))

### Create `transfers_ft.txt` 


In [ ]:
xfer_dict = gtfs_plus.create_tranfers(schedule,max_xfer_dist=0.6)

with open(os.path.join(OUTPUT_DIR,'transfers_ft.txt'),'wb') as f:
    f.write("from_stop_id,to_stop_id,dist\n")
    for k,v in xfer_dict.iteritems():
        f.write("%s,%s,%4.2f\n" % (k[0],k[1],v))
        #and reverse link
        f.write("%s,%s,%4.2f\n" % (k[1],k[0],v))

### `walk_access.txt` 
For this single-trip example, let's create walk access on the fly using simple crow-fly distance

## Define a single demand trip
For now, select an origin and destination `stop_id`.  Your trip will begin and end at these stops w/out walk access to other trips for now.

In [ ]:
ORIGIN_STOP      = "776415"
DESTINATION_STOP = "777546"

with open(os.path.join(OUTPUT_DIR,'walk_access_ft.txt'),'wb') as f:
    f.write("taz,stop_id,direction,dist\n")
    f.write("t_%s,%s,access,0.01\n" % (ORIGIN_STOP, ORIGIN_STOP))
    f.write("t_%s,%s,egress,0.01\n" % (DESTINATION_STOP, DESTINATION_STOP))

In [ ]:
with open(os.path.join(OUTPUT_DIR,'trip_list.txt'),'wb') as f:
    f.write("person_id,person_trip_id,o_taz,d_taz,mode,purpose,departure_time,arrival_time,time_target,vot\n")
    f.write("Tuffie,1,t_%s,t_%s,transit,appcon,7:00:00,8:00:00,departure,10\n" % (ORIGIN_STOP, DESTINATION_STOP))

In [ ]:
ACCESS_WEIGHT      = 2.0
EGRESS_WEIGHT      = 2.0
WAIT_TIME_WEIGHT   = 2.0
IVT_WEIGHT         = 1.0
XFER_PEN_WEIGHT    = 5
XFER_WALK_WEIGHT   = 2

with open(os.path.join(OUTPUT_DIR,'pathweight_ft.txt'),'wb') as f:
    f.write("user_class,purpose,demand_mode_type,demand_mode,supply_mode,weight_name,weight_value\n")
    f.write("all,appcon,access,walk,walk_access,time_min,%4.2f\n" % (ACCESS_WEIGHT))
    f.write("all,appcon,egress,walk,walk_egress,time_min,%4.2f\n" % (EGRESS_WEIGHT))
    f.write("all,appcon,transit,transit,local_bus,wait_time_min,%4.2f\n" % (WAIT_TIME_WEIGHT))
    f.write("all,appcon,transit,transit,local_bus,in_vehicle_time_min,%4.2f\n" % (IVT_WEIGHT))
    f.write("all,appcon,transfer,transfer,transfer,transfer_penalty,%4.2f\n" % (XFER_PEN_WEIGHT ))
    f.write("all,appcon,transfer,transfer,transfer,walk_time_min,%4.2f\n" % (XFER_WALK_WEIGHT))

# Run Fast-Trips !


In [ ]:
from fasttrips import Run

In [ ]:
INPUT_NETWORKS   = OUTPUT_DIR
INPUT_DEMAND     = OUTPUT_DIR
INPUT_WEIGHTS    = os.path.join(OUTPUT_DIR,'pathweight_ft.txt')
RUN_CONFIG       = os.path.join(OUTPUT_DIR,'config_ft.txt')
OUTPUT_FOLDER    = r"FT_TEST_SINGLE_DEMAND"
OUTPUT_DIR       = OUTPUT_DIR
ITERATIONS       = 1
PATHFINDING_TYPE = "stochastic"
OVERLAP          = "None"
DISPERSION       = 0.5

In [ ]:
Run.run_fasttrips(input_network_dir= INPUT_NETWORKS,
                  input_demand_dir = INPUT_DEMAND,
                  run_config       = RUN_CONFIG,
                  input_weights    = INPUT_WEIGHTS,
                  output_dir       = OUTPUT_DIR,
                  output_folder    = OUTPUT_FOLDER,
                  pathfinding_type = PATHFINDING_TYPE,
                  iters            = ITERATIONS,
                  overlap_variable = OVERLAP,
                  dispersion       = DISPERSION)

# Examine Results

In [ ]:
from bokeh.charts import Bar, Histogram, TimeSeries, output_file, Line
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

## Vehicle/Route Level

In [ ]:
full_output_directory=os.path.join(OUTPUT_DIR,OUTPUT_FOLDER)
vehicles_df = pd.read_csv(os.path.join(full_output_directory,"veh_trips.csv"), 
                                       sep=",", 
                                       parse_dates=['arrival_time', 'departure_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )
#vehicles_df.head()

In [ ]:
max_iteration     = vehicles_df['iteration'].max()
max_pf_iteration  = vehicles_df['pathfinding_iteration'].max()

final_vehicles_df = vehicles_df[(vehicles_df['iteration'] == max_iteration) & (vehicles_df['pathfinding_iteration']==max_pf_iteration)]

In [ ]:
tooltips=[
    ('Trip ID', '@trip_id'),
]

all_lines  = Bar(final_vehicles_df, "route_id", values='boards', stack='trip_id',title="Boardings by Route",legend=False,
            xlabel="Route ID", ylabel="Boardings",tooltips=tooltips)

ridership = show(all_lines, notebook_handle=True)

## Person Level

In [ ]:
chosen_df = pd.read_csv(os.path.join(full_output_directory,"chosenpaths_links.csv"), 
                                       sep=",", 
                                       parse_dates=['new_A_time', 'new_B_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )



In [ ]:
pline = Line(chosen_df, x="linknum", y="new_A_time", color="linkmode",title="line", width=10)


show(pline, notebook_handle=True)


In [ ]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid,DatetimeAxis, LabelSet
from bokeh.models.glyphs import Segment
from bokeh.io import curdoc, show



palette = ["#d73838", "#dea23a", "#b1d149", "#5d85b5", "#9d56a4",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]

mode = list(set(chosen_df["mode"]))
l    = len(mode)
mode_color = dict(zip(mode,palette[0:l]))


def assignColorByMode(mode):
    if mode in mode_color.keys():
        return mode_color[mode]
    return "#f0f0f0"
    
chosen_df["color"]= map(assignColorByMode, chosen_df["mode"])

def createAnnotation(mode,sim_cost,route_id,trip_id):
    return str(mode)+" "+str(sim_cost)+" "+str(route_id)+":"+str(trip_id)

chosen_df["annotation"]= map(createAnnotation, chosen_df["mode"],chosen_df["sim_cost"],chosen_df["route_id"],chosen_df["trip_id"])

source = ColumnDataSource(chosen_df)

In [ ]:
xdr = DataRange1d()
ydr = DataRange1d()

plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=800, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(y0="linknum", x0="new_A_time", y1="linknum", x1="new_B_time", line_color="color", line_width=20)
plot.add_glyph(source, glyph)

xaxis = DatetimeAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

mode_labels = LabelSet(x="new_A_time", y="linknum", text="mode", y_offset=-5,x_offset=10,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='left')
plot.add_layout(mode_labels)

curdoc().add_root(plot)

show(plot, notebook_handle=True)

## Path Choice

In [ ]:
PATHS  = r"pathset_paths.csv"

pathfile_df = pd.read_csv(os.path.join(full_output_directory,PATHS), sep=",")
pd.set_option('display.max_colwidth',160) #widen so you can see whole description
pathfile_df[["person_id","person_trip_id","pathnum","description","sim_cost","logsum","probability"]]

### Display links
Use the link file to display each component of the path and their respective costs.

In [ ]:
LINKS  = r"pathset_links.csv"
linkfile_df = pd.read_csv(os.path.join(full_output_directory,LINKS),
                                       sep=",", 
                                       parse_dates=['new_A_time', 'new_B_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

## Add Fast-Trips path probability information to Fast-Trips link file
linkfile_df = pd.merge(left  = linkfile_df,
                       right = pathfile_df,
                       how   = "left",
                       left_on = ["person_id","person_trip_id","pathnum","chosen","missed_xfer"],
                       right_on = ["person_id","person_trip_id","pathnum","chosen","missed_xfer"])

#linkfile_df.head()
#linkfile_df.columns

In [ ]:

maxlinknum = linkfile_df["linknum"].max()
    
linkfile_df["color"]= map(assignColorByMode, linkfile_df["mode"])

def createAnnotation(mode,sim_cost,route_id,trip_id,probability):
    return str(mode)+" "+str(sim_cost)+" "+str(route_id)+":"+str(trip_id)

linkfile_df["annotation"]= map(createAnnotation, linkfile_df["mode"],linkfile_df["sim_cost_x"],linkfile_df["route_id"],linkfile_df["trip_id"],linkfile_df["probability"])

def yloc(pathnum,linknum):
    return (pathnum*(maxlinknum+1))+linknum

linkfile_df["yloc"]= map(yloc, linkfile_df["pathnum"],linkfile_df["linknum"])

linksource = ColumnDataSource(linkfile_df)
#linkfile_df

In [ ]:
xdr2 = DataRange1d()
ydr2 = DataRange1d()

choiceplot = Plot(
    title=None, x_range=xdr2, y_range=ydr2, plot_width=800, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(y0="yloc", x0="new_A_time", y1="yloc", x1="new_B_time", line_color="color", line_width="sim_cost_x")
choiceplot.add_glyph(linksource, glyph)

choicexaxis = DatetimeAxis()
choiceplot.add_layout(choicexaxis, 'below')

choiceplot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))

choice_labels = LabelSet(x="new_A_time", y="yloc", text="annotation", y_offset=-5,x_offset=10,
                  text_font_size="8pt", text_color="#555555",
                  source=linksource, text_align='left')
choiceplot.add_layout(choice_labels)

curdoc().add_root(choiceplot)

##TODO add nodes
##TODO make annotations more visible; remove fields that aren't necessary for that link type
##TODO standardize colors by mode
##TODO add probability

show(choiceplot, notebook_handle=True)